In [32]:
%pip install selenium
%pip install webdriver-manager
%pip install bs4
%pip install pandas

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException
from google.cloud import storage
from datetime import datetime
import pandas as pd
import os
import re
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# Function to generate Parquet filename
def generate_parquet_filename(search):
    date_string = datetime.now().strftime("%Y_%m_%d")
    clean_search = re.sub(r'[^a-zA-Z0-9]', '', search).lower()
    now = datetime.utcnow()
    epoch = datetime(1970, 1, 1)
    seconds_since_epoch = str(int((now - epoch).total_seconds()))
    parquet_filename = f'{date_string}__{seconds_since_epoch}__{clean_search}.parquet'
    return parquet_filename

# Function to upload the Parquet file to GCS
def upload_to_gcs(dataframe, bucket_name, folder_name, file_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f"{folder_name}/{file_name}")

    if blob.exists():
        blob.delete()
        print(f"File '{file_name}' already exists. Deleting the old file.")

    temp_file = f"/tmp/{file_name}"
    dataframe.to_parquet(temp_file, index=False)
    blob.upload_from_filename(temp_file)
    os.remove(temp_file)
    print(f"File '{file_name}' uploaded to GCS bucket '{bucket_name}' in folder '{folder_name}'.")

# Function to delete a file from GCS
def delete_from_gcs(bucket_name, folder_name, file_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f"{folder_name}/{file_name}")

    if blob.exists():
        blob.delete()
        print(f"File '{file_name}' deleted from GCS bucket '{bucket_name}'.")
    else:
        print(f"File '{file_name}' does not exist in GCS bucket '{bucket_name}'.")

# Function to download a file from GCS
def download_from_gcs(bucket_name, folder_name, file_name, local_file):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f"{folder_name}/{file_name}")
    blob.download_to_filename(local_file)
    print(f"File '{file_name}' downloaded from GCS bucket '{bucket_name}' to '{local_file}'.")

# Download existing Parquet files from GCS
def download_existing_parquets(bucket_name, folder_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    parquet_files = [file.name for file in bucket.list_blobs(prefix=folder_name) if file.name.endswith('.parquet')]

    existing_dfs = []
    for file_name in parquet_files:
        local_file = f"/tmp/{os.path.basename(file_name)}"
        download_from_gcs(bucket_name, folder_name, os.path.basename(file_name), local_file)
        df = pd.read_parquet(local_file)
        existing_dfs.append(df)
        os.remove(local_file)  # Clean up local file

    if existing_dfs:
        combined_df = pd.concat(existing_dfs, ignore_index=True)
    else:
        combined_df = pd.DataFrame()

    return combined_df

# Set up WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Open the website
url = 'https://www.apptweak.com/en/free-tools/keyword-auto-suggestions'
driver.get(url)

# Wait for the page to load
wait = WebDriverWait(driver, 10000000)  # Increased timeout to 10000000 seconds

# Get today's date in format YYYY_MM_DD
today_date = datetime.today().strftime('%Y_%m_%d')

# Initialize a list to collect the data
bucket_name = 'apptweak_scraper'
folder_name = 'first_batch_of_searches'
batch_folder_name = f'{folder_name}/{today_date}'

# Static search term
search = 'a'

# Download existing Parquet files
existing_df = download_existing_parquets(bucket_name, batch_folder_name)

# Function to select options from a dropdown
def select_from_dropdown(dropdown_id, option_text):
    try:
        select = Select(wait.until(EC.presence_of_element_located((By.ID, dropdown_id))))
        select.select_by_visible_text(option_text)
        print(f"Selected '{option_text}' from dropdown '{dropdown_id}'.")
    except TimeoutException:
        print(f"Timeout waiting for dropdown '{dropdown_id}' or option '{option_text}' to be present.")
    except Exception as e:
        print(f"Error selecting '{option_text}' from dropdown '{dropdown_id}': {e}")

# Function to get options from a dropdown
def get_dropdown_options(dropdown_id):
    try:
        select = Select(wait.until(EC.presence_of_element_located((By.ID, dropdown_id))))
        options = [option.text for option in select.options]
        print(f"Available options for dropdown '{dropdown_id}': {options}")
        return options
    except TimeoutException:
        print(f"Timeout waiting for dropdown '{dropdown_id}' to be present.")
        return []

# Get the list of countries and stores from the dropdown
country_options = get_dropdown_options('country')
store_options = get_dropdown_options('store')

# Limit to the first 5 countries
country_options = country_options[:5]

# Initialize a list to collect data for the search term 'a'
aggregated_data = []

# Iterate over the limited list of countries
for country_name in country_options:
    print(f"Processing: Country='{country_name}'")

    # Select the country
    select_from_dropdown('country', country_name)

    # Iterate over available stores
    for store_name in store_options:
        print(f"Processing: Store='{store_name}'")

        # Select the store
        select_from_dropdown('store', store_name)

        # Fetch language options dynamically after selecting the store
        language_options = get_dropdown_options('language')

        for language_name in language_options:
            print(f"Processing: Language='{language_name}'")

            # Wait for the 'keyword' input field to be available and enter the letter 'a'
            try:
                keyword_input = wait.until(EC.presence_of_element_located((By.ID, 'keyword')))
                keyword_input.clear()
                keyword_input.send_keys(search)
                keyword_input.send_keys(Keys.RETURN)
            except TimeoutException:
                print(f"Timeout waiting for the 'keyword' input field.")
                continue

            # Select the language
            select_from_dropdown('language', language_name)

            # Wait for the progress bar to disappear
            try:
                wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '.progress-bar__inner')))
            except TimeoutException:
                print(f"Progress bar did not disappear within 60 seconds for search '{search}'.")

            # Explicitly wait for the "Suggest" button to be clickable
            try:
                suggest_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.js-keyword-auto-suggestion')))

                for _ in range(3):  # Retry up to 3 times
                    try:
                        suggest_button.click()
                        break
                    except ElementClickInterceptedException:
                        time.sleep(2)
                        print(f"Timing for Search '{search}'...")
            except TimeoutException:
                print(f"Timeout waiting for the 'Suggest' button to be clickable.")

            # Wait until the search results section becomes visible
            try:
                results_section = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.free-tools__kw-lists-container')))

                # Extract all <ul> elements for the current search
                ul_elements = results_section.find_elements(By.CSS_SELECTOR, 'ul')

                for ul in ul_elements:
                    li_elements = ul.find_elements(By.CSS_SELECTOR, '.free-tools__kw.listing-k-suggest-t')

                    if len(li_elements) > 1:
                        for rank, li in enumerate(li_elements[1:], start=1):  # Skip the first <li>
                            suggestion = li.text
                            words = suggestion.split()
                            if len(words) > 1:
                                next_word_letter = words[1][0].lower()
                            else:
                                next_word_letter = None

                            aggregated_data.append([search, store_name, country_name, language_name, next_word_letter, suggestion, rank, url, datetime.now()])
            except TimeoutException:
                print(f"Timeout waiting for the search results section to become visible.")

# Create a DataFrame for the aggregated data
new_df = pd.DataFrame(aggregated_data, columns=['search', 'store', 'country', 'language', 'next_word_letter', 'suggestion', 'rank', 'scraping_url', 'scraping_timestamp'])

# Append new data to existing data
combined_df = pd.concat([existing_df, new_df], ignore_index=True)

# Generate the Parquet filename for the updated data
parquet_filename = generate_parquet_filename(search)

# Remove old Parquet files from GCS
parquet_files_to_delete = [file for file in existing_df.columns if file.endswith('.parquet')]
for file in parquet_files_to_delete:
    delete_from_gcs(bucket_name, batch_folder_name, os.path.basename(file))

# Upload the updated DataFrame as a Parquet file to GCS
upload_to_gcs(combined_df, bucket_name, batch_folder_name, parquet_filename)

# Close the driver
driver.quit()


File '2024_09_19__1726769391__a.parquet' downloaded from GCS bucket 'apptweak_scraper' to '/tmp/2024_09_19__1726769391__a.parquet'.
Available options for dropdown 'country': ['United Arab Emirates', 'Afghanistan', 'Albania', 'Armenia', 'Angola', 'Argentina', 'Austria', 'Australia', 'Bosnia And Herzegovina', 'Belgium', 'Bulgaria', 'Brazil', 'Botswana', 'Belarus', 'Canada', 'Congo, The Democratic Republic Of The', 'Switzerland', "Côte D'Ivoire", 'Chile', 'Cameroon', 'China', 'Colombia', 'Cyprus', 'Czech Republic', 'Germany', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Estonia', 'Egypt', 'Spain', 'Finland', 'France', 'Gabon', 'United Kingdom', 'Grenada', 'Georgia', 'Ghana', 'Greece', 'Hong Kong', 'Croatia', 'Hungary', 'Indonesia', 'Ireland', 'Israel', 'India', 'Iraq', 'Iceland', 'Italy', 'Jordan', 'Japan', 'Kenya', 'Cambodia', 'Korea, Republic Of', 'Kuwait', 'Kazakstan', "Lao People's Democratic Republic", 'Lebanon', 'Lithuania', 'Luxembourg', 'Latvia', 'Libya', 'Morocco', 'Mo

In [39]:
from google.cloud import bigquery, storage

# Initialize clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

# Define your parameters
project_id = "jaber-financial"
dataset_id = "financial_data"
bucket_name = "apptweak_scraper"
apptweak_dir_prefix = "first_batch_of_searches/2024_09_19/"

def list_parquet_files(bucket_name, prefix):
    """List all parquet files in a specific GCS bucket and prefix."""
    bucket = gcs_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.parquet')]

# Load all files from data/history/ directory into a single table named 'companies_details'
apptweak_details_files = list_parquet_files(bucket_name, apptweak_dir_prefix)

apptweak_table_ref = bq_client.dataset(dataset_id).table("apptweak_2024_09_19")
apptweak_job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,  # Parquet file format
    autodetect=True
)

# Start the load job for history files
load_job = bq_client.load_table_from_uri(
    [f"gs://{bucket_name}/{file}" for file in apptweak_details_files],
    apptweak_table_ref,
    job_config=apptweak_job_config
)

# Wait for the job to complete
load_job.result()

# Check the result
apptweak_details_table = bq_client.get_table(apptweak_table_ref)
print(f"Loaded {apptweak_details_table.num_rows} rows into {dataset_id}:apptweak_2024_09_19.")


Loaded 5064 rows into financial_data:apptweak_2024_09_19.
